In [1]:
from collections import defaultdict as ddict
def statistic_files(files,delta=10e-6):
    pc_num=0
    spect_df=pd.DataFrame(columns=['m/z','inten','peaks'])
    stat_df=pd.DataFrame(columns=['files','all_spect_num','all_pos_ms2_spect_num','184_spect_num'])
    for file in files:
        ser_dd=ddict(int)
        print(f'in the {file}')
        ser_dd['files']=file
        TIC=pymzml.run.Reader(file)
        total_num=TIC.get_spectrum_count()
        ser_dd['all_spect_num']=total_num
        ser_dd['all_pos_ms2_spect_num']=0
        ser_dd['184_spect_num']=0
        
        for spectrum in TIC:
            #print(spectrum.ID,type(spectrum.ID))
            #print(f"handling the {spectrum.ID}th /{total_num} spectrum")
            if spectrum['MS:1000129'] or spectrum['MS:1000511']==1:
                continue
            ser_dd['all_pos_ms2_spect_num']+=1
            if spectrum.has_peak(184.07294):
                current_mz=spectrum['MS:1000744']
                current_inten=spectrum['MS:1000042']
                current_inten_peaks=spectrum.peaks("centroided")
                output=[current_mz,current_inten,current_inten_peaks]
                ser_dd['184_spect_num']+=1
                if spect_df.empty:
                    spect_df.loc[len(spect_df),:]=output
                    pc_num+=1
                    continue
                if  abs( (spect_df['m/z']-current_mz)/current_mz).min()>delta:
                    spect_df.loc[len(spect_df),:]=output
                    pc_num+=1
                else:#update
                    hit=(abs(spect_df['m/z']-current_mz) == abs(spect_df['m/z']-current_mz).min())

                    #print(hit,f'\n the hit num is {hit.sum()}\n',spect_df.loc[hit,:])
                    if (spect_df.loc[hit,'inten']<current_inten).all():
                        spect_df.loc[hit,:]=output

                        
        stat_df=stat_df.append(ser_dd,ignore_index=True)
        #if len(stat_df)>3:break;
        #break;



    return spect_df,stat_df

def compute_spectrum(raw_file_path,output_ms2_path,output_statistics_path):
    import pandas as pd
    import os
    para=pd.read_excel(raw_file_path)
    mzml_file=''
    for idx,content in para.iterrows():
        mzml_file=os.path.join(content['dir'],content['files'])
    mzml_files=[]
    for idx,content in para.iterrows():
        mzml_files.append( os.path.join(content['dir'],content['files']) )
    mzml_files

    all_files_pc_df,the_stat_df=statistic_files(mzml_files)
    print(all_files_pc_df.shape,the_stat_df.shape)

    import pickle

    with open(output_ms2_path','wb') as f:
        pickle.dump(all_files_pc_df,f)
    the_stat_df.to_excel(output_statistics_path)
    


In [ ]:
compute_spectrum(r'./mzML_files.xlsx',r'./temp_all_pc3.pc',r'./temp_stastic_ms2.xlsx')